# import all packages

In [1]:
# Import all the things we need ---
#   by setting env variables before Keras import you can set up which backend and which GPU it uses
import os,random
import keras
#os.environ["KERAS_BACKEND"] = "theano"

os.environ["KERAS_BACKEND"] = "tensorflow"
os.environ["CUDA_VISIBLE_DEVICE"]  = '0'
#os.environ["THEANO_FLAGS"]  = "floatX=float32"
#os.environ["THEANO_FLAGS"]  = "device=cuda%d"%(1)

Using TensorFlow backend.


In [2]:
import numpy as np
import seaborn as sns
import pickle, random, sys

In [3]:
from keras.utils import np_utils
import keras.models as models
from keras.layers.core import Reshape,Dense,Dropout,Activation,Flatten
from keras.layers.noise import GaussianNoise
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.regularizers import *
#from keras.optimizers import adam
from tensorflow.keras.optimizers import Adam
#import theano as th
#import theano.tensor as T
import os
WEIGHTS_PATH = ('resnet_like_weights_tf_dim_ordering_tf_kernels.h5')
from keras.models import Model
from keras.layers import Input,Dense,Conv1D,MaxPool1D,ReLU,Dropout,Softmax
from keras.layers import LSTM
import keras
from keras.callbacks import LearningRateScheduler
import keras.backend as K
from keras.callbacks import LearningRateScheduler
from keras.regularizers import *
from tensorflow.keras.optimizers import Adam
from keras.models import model_from_json
import tensorflow as tf
import importlib,sys

importlib.reload(sys)

from mpl_toolkits.mplot3d import Axes3D

from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Load the dataset ...
#  You will need to seperately download or generate this file
dbfile = open('RML2016.10a_dict.dat', 'rb')      
Xd = pickle.load(dbfile,encoding='latin1') 
snrs,mods = map(lambda j: sorted(list(set(map(lambda x: x[j], Xd.keys())))), [1,0])
X = []  
lbl = []
for mod in mods:
    for snr in snrs:
        X.append(Xd[(mod,snr)])
        for i in range(Xd[(mod,snr)].shape[0]):  lbl.append((mod,snr))
X = np.vstack(X)

In [4]:
# Partition the data
#  into training and test sets of the form we can train/test on 
#  while keeping SNR and Mod labels handy for each
np.random.seed(2016)
n_examples = X.shape[0]
n_train = int(n_examples * 0.9)
train_idx = np.random.choice(range(0,n_examples), size=n_train, replace=False)
test_idx = list(set(range(0,n_examples))-set(train_idx))
X_train = X[train_idx]
X_test =  X[test_idx]
def to_onehot(yy):
    yy1 = np.zeros([len(yy), max(yy)+1])
    yy1[np.arange(len(yy)),yy] = 1
    return yy1
Y_train = to_onehot(list(map(lambda x: mods.index(lbl[x][0]), train_idx)))
Y_test = to_onehot(list(map(lambda x: mods.index(lbl[x][0]), test_idx)))

In [5]:
print('数据集总数：',n_examples)
print('调制方式' , len(mods),'种:' ,mods)
print('信噪比:',snrs)

数据集总数： 220000
调制方式 11 种: ['8PSK', 'AM-DSB', 'AM-SSB', 'BPSK', 'CPFSK', 'GFSK', 'PAM4', 'QAM16', 'QAM64', 'QPSK', 'WBFM']
信噪比: [-20, -18, -16, -14, -12, -10, -8, -6, -4, -2, 0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


In [6]:
# Y_train = np.argmax(Y_train, axis=-1)
# Y_test = np.argmax(Y_test, axis=-1)

# FEAT_DATA_LOAD

In [7]:
X_train_feature_expert = pd.read_csv('X_train_feature_expert.csv',header=None).drop(columns=[0],index=[0])
X_test_feature_expert = pd.read_csv('X_test_feature_expert.csv',header=None).drop(columns=[0],index=[0])

X_train_feature_time = pd.read_csv('X_train_feature_time.csv',header=None).drop(columns=[0],index=[0])
X_test_feature_time = pd.read_csv('X_test_feature_time.csv',header=None).drop(columns=[0],index=[0])

X_train_feature_old = pd.read_csv('X_train_feature_old.csv',header=None).drop(columns=[0],index=[0])
X_test_feature_old = pd.read_csv('X_test_feature_old.csv',header=None).drop(columns=[0],index=[0])

In [32]:
# concate time feature and complex feature together
# X_train_feature = np.concatenate((X_train_feature_time, X_train_feature_complex), axis = 1)
# X_test_feature = np.concatenate((X_test_feature_time, X_test_feature_complex), axis = 1)
#choose1
# X_train_feature = X_train_feature_complex
# X_test_feature = X_test_feature_complex
#choose1
X_train_feature = X_train_feature_time
X_test_feature = X_test_feature_time

print(X_train_feature.shape)
print(X_test_feature.shape)

(198000, 132)
(22000, 132)


In [33]:
# standardize the features

from sklearn.feature_selection import VarianceThreshold
X_all = np.concatenate((X_train_feature, X_test_feature), axis = 0)

s0 = X_train_feature.shape[0]
s1 = X_train_feature.shape[1]

k = 1
interval = int(s1/k)
X_train_std_all_list = []
X_test_std_list = []
sc_train = []
for i in range(k):
    X_all_filter = VarianceThreshold(threshold=(0)).fit_transform(X_all[:,interval*i: interval*(i+1)])
    sc_train.append(StandardScaler())
    print(X_all_filter.shape)
    sc_train[i].fit(X_all_filter)
    X_train_std_all_list.append(sc_train[i].transform(X_all_filter[:s0,:])) 
    X_test_std_list.append(sc_train[i].transform(X_all_filter[s0:,:]))
X_train_std_all = np.concatenate((X_train_std_all_list), axis = 1)
X_test_std = np.concatenate((X_test_std_list), axis = 1)

print("X_train_std_all,", X_train_std_all.shape)
print("X_test_std,", X_test_std.shape)

(220000, 127)
X_train_std_all, (198000, 127)
X_test_std, (22000, 127)


In [34]:
# devide train and val data
from sklearn.model_selection import train_test_split
X_train_std, X_val_std, y_train, y_val = train_test_split(X_train_std_all, Y_train, test_size=0.111, random_state=42)
print("X_train_std,", X_train_std.shape)
print("X_val_std,", X_val_std.shape)
print("X_test_std,", X_test_std.shape)

X_train_std, (176022, 127)
X_val_std, (21978, 127)
X_test_std, (22000, 127)


In [35]:
X_train_std = pd.DataFrame(X_train_std)
X_val_std = pd.DataFrame(X_val_std)
X_test_std = pd.DataFrame(X_test_std)

In [36]:
# inf,nan数据填充
X_train_std = (X_train_std.replace([np.inf, -np.inf], np.nan)).fillna(value = 0) 
X_val_std = (X_val_std.replace([np.inf, -np.inf], np.nan)).fillna(value = 0) 
X_test_std = (X_test_std.replace([np.inf, -np.inf], np.nan)).fillna(value = 0) 

In [37]:
X_train_std.shape,X_test_std.shape,X_val_std.shape

((176022, 127), (22000, 127), (21978, 127))

# load_trained_models

In [38]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
import numpy as np
from matplotlib import pyplot as plt

from art.estimators.classification import KerasClassifier
from art.attacks.evasion import FastGradientMethod, CarliniLInfMethod,ProjectedGradientDescent,UniversalPerturbation,CarliniL2Method

import joblib
from tensorflow.keras.models import load_model
# model_cnn = load_model('model_saved\Feat_based\CNN_basic.h5')
# model_xgb = joblib.load('model_saved\Feat_based\XGB_FEAT_stat.pkl')
# model_cart = joblib.load('model_saved\Feat_based\CART_FEAT_basic.pkl')
# model_lgb = joblib.load('model_saved\Feat_based\LightGBM_basic.pkl')

# model_cnn = load_model('model_saved\Feat_based\CNN_expert_200.h5')
# model_xgb = joblib.load('model_saved\Feat_based\XGB_FEAT_expert.pkl')
# model_cart = joblib.load('model_saved\Feat_based\CART_FEAT_expert.pkl')
# model_lgb = joblib.load('model_saved\Feat_based\LightGBM_expert.pkl')

model_cnn = load_model('model_saved\Feat_based\CNN_time_200.h5')
model_xgb = joblib.load('model_saved\Feat_based\XGB_FEAT_time.pkl')
model_cart = joblib.load('model_saved\Feat_based\CART_FEAT_time.pkl')
model_lgb = joblib.load('model_saved\Feat_based\LightGBM_time.pkl')

# 基于CNN代理模型的黑盒攻击

In [43]:
# ① 提取SNR = 10dB的测试样本
def extract_snr_10dB(X_test,Y_test,snr = 10):
    #snr = 10
    test_SNRs = list(map(lambda x: lbl[x][1], test_idx))
    test_X_i = X_test[np.where(np.array(test_SNRs)==snr)]
    test_Y_i = Y_test[np.where(np.array(test_SNRs)==snr)]  
    return test_X_i,test_Y_i

# ② 生成代理模型对抗样本
def generate_AEs(model, EPS, attack_samples, attack = 'FGSM'):
    '''
    基于代理模型，构建对抗扰动
    输出：基于代理模型的对抗样本
    '''
    classifier = KerasClassifier(model=model)
    #②定义攻击算法
    if attack == 'FGSM':
        attacker = FastGradientMethod(estimator=classifier, eps = EPS)
        
    elif attack == 'PGD':
        attacker = ProjectedGradientDescent(estimator=classifier, eps=EPS,max_iter=100)
        
    elif attack == 'UAP':
        attacker = UniversalPerturbation(classifier = classifier, eps = EPS, attacker = 'fgsm')
        
    elif attack == 'CW':
        attacker = CarliniL2Method(classifier = classifier, confidence = 0)
        
    elif attack == 'ZOO':
        attacker = ZooAttack(classifier=classifier, confidence=0.0, targeted=False, learning_rate=1e-1, max_iter=20,
                    binary_search_steps=10, initial_const=1e-3, abort_early=True, use_resize=False, 
                    use_importance=False, nb_parallel=2, batch_size=1, variable_h=0.2)
    #③生成对抗样本
    x_test_adv = attacker.generate(attack_samples)

    return x_test_adv

classes = mods

# ③模型黑盒攻击，输出SNR=10dB各模型攻击结果
def attack_results(test_X_i,test_Y_i,model):
    test_Y_i_hat = model.predict(test_X_i)
    conf = np.zeros([len(classes),len(classes)])
    confnorm = np.zeros([len(classes),len(classes)])
    for i in range(0,test_X_i.shape[0]):
        j = list(test_Y_i[i,:]).index(1)
        k = int(np.argmax(test_Y_i_hat[i,:]))
        conf[j,k] = conf[j,k] + 1
#         for i in range(0,len(classes)):
#             confnorm[i,:] = conf[i,:] / np.sum(conf[i,:])
    #plt.figure()
    #plot_confusion_matrix(confnorm, labels=classes, title="ConvNet Confusion Matrix (SNR=%d)"%(snr))
    cor = np.sum(np.diag(conf))
    ncor = np.sum(conf) - cor
    #print("Overall Accuracy: ", cor / (cor+ncor))
    acc = 1.0*cor/(cor+ncor)
    return acc
    #print(acc)

In [40]:
# ① 提取SNR = 10dB的测试样本
X_test = X_test_std.values.reshape(X_test_std.shape[0],X_test_std.shape[1],1)
test_X_10db, test_Y_10db = extract_snr_10dB(X_test,Y_test)

In [41]:
model_cnn_acc = []
model_cart_acc = []
model_xgb_acc = []
model_lgb_acc = []

for i in range(1,2):
    eps = 0.01*i
    
    # ② 生成代理模型对抗样本
    x_test_adv = generate_AEs(model_cnn,eps,test_X_10db, attack = 'CW')
    x_test_adv_2d = x_test_adv.reshape(x_test_adv.shape[0],x_test_adv.shape[1])
    
    # ③模型黑盒攻击，输出SNR=10dB各模型攻击结果
    model_cnn_acc.append(attack_results(x_test_adv,test_Y_10db,model_cnn))
    model_cart_acc.append(attack_results(x_test_adv_2d,test_Y_10db,model_cart))
    model_xgb_acc.append(attack_results(x_test_adv_2d,test_Y_10db,model_xgb))
    model_lgb_acc.append(attack_results(x_test_adv_2d,test_Y_10db,model_lgb))

C&W L_2:   0%|          | 0/1066 [00:00<?, ?it/s]

In [42]:
model_cnn_acc,model_cart_acc,model_xgb_acc,model_lgb_acc 

([0.4540337711069418],
 [0.5722326454033771],
 [0.723264540337711],
 [0.702626641651032])

In [132]:
# ④ 攻击结果写入excel
import pandas as pd
writer = pd.ExcelWriter('ATT_CW_expert.xlsx')		# 写入Excel文件

pd.DataFrame(model_cnn_acc).to_excel(writer, 'model_cnn_acc', float_format='%.5f')
pd.DataFrame(model_cart_acc).to_excel(writer, 'model_cart_acc', float_format='%.5f')
pd.DataFrame(model_xgb_acc).to_excel(writer, 'model_xgb_acc', float_format='%.5f')
pd.DataFrame(model_lgb_acc).to_excel(writer, 'model_lgb_acc', float_format='%.5f')
writer.save()

writer.close()

# **ART内置模型训练方式实现**

In [53]:
y_train

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# lightgbm

In [ ]:
import lightgbm as lgb
from art.estimators.classification import LightGBMClassifier

# Step 2: Create the model

params = {"objective": "multiclass", "metric": "multi_logloss", "num_class": 11}
train_set = lgb.Dataset(X_train_std.values, label=y_train)
eval_set = lgb.Dataset(X_val_std.values, label=y_val)
model = lgb.train(params=params, train_set=train_set, num_boost_round=100, valid_sets=[eval_set])

# cart

In [ ]:
from sklearn import datasets,model_selection,metrics,tree,preprocessing
from art.attacks.evasion import DecisionTreeAttack

#x=preprocessing.StandardScaler().fit_transform(x)
# 划分训练集、测试集
#x_train,x_test,y_train,y_test=model_selection.train_test_split(X_train_std,y_train,test_size=0.3)
# 导入决策树
model=tree.DecisionTreeClassifier(max_depth=5)
# 模型训练
model.fit(X_train_std.values,y_train)
# 模型预测
y_pred=model.predict(X_val_std.values)
# 模型性能评价
print(metrics.accuracy_score(y_pred,y_val))

# xgboost

In [ ]:
from art.estimators.classification import XGBoostClassifier
import xgboost as xgb

In [ ]:
num_round = 10
param = {'objective': 'multi:softprob', 'metric': 'multi_logloss', 'num_class': 11}
train_data = xgb.DMatrix(X_train_std.values, label=y_train)
eval_data = xgb.DMatrix(X_val_std.values, label=y_val)
evallist = [(eval_data, 'eval'), (train_data, 'train')]
model = xgb.train(param, train_data, num_round, evallist)

In [20]:
def extract_10dB(X_test_std,Y_test):
    snr = 10
    test_SNRs = list(map(lambda x: lbl[x][1], test_idx))
    test_X_i = X_test_std[np.where(np.array(test_SNRs)==snr)]
    test_Y_i = Y_test[np.where(np.array(test_SNRs)==snr)]   
    return test_X_i,test_Y_i

In [67]:
# 对抗样本生成
from art.estimators.classification import SklearnClassifier
from art.attacks.evasion import ZooAttack
from sklearn.metrics import accuracy_score
from art.attacks.evasion import ProjectedGradientDescent,FastGradientMethod

from art.attacks.evasion import CarliniL2Method

def get_adversarial_examples(model, x_test, y_test, EPS, model_type = 'lgb',attack = 'PGD'):
    
    test_X_i, test_Y_i = extract_10dB(x_test,y_test)
    
    if model_type == 'xgb':
        # Create ART classifier for XGBoost
        classifier = XGBoostClassifier(model=model, nb_features=test_X_i.shape[1], nb_classes=11)
        
    elif model_type == 'cart':
        classifier = SklearnClassifier(model=model)
        
    elif model_type == 'lgb':
        classifier = LightGBMClassifier(model=model)
        
        
    # Create ART attack
    if attack == 'FGM':
        attacker = FastGradientMethod(estimator=art_classifier, eps=EPS)
        
    if attack == 'PGD':
        attacker = ProjectedGradientDescent(estimator=art_classifier, norm=1, eps=EPS, eps_step=0.1,
                                            max_iter=10, targeted=False, num_random_init=0, batch_size=1,
                                            verbose=False)
    elif attack == 'CW':
        attacker = CarliniL2Method(classifier=art_classifier, max_iter=20, verbose=False)
    
    elif attack == 'ZOO':
        attacker = ZooAttack(classifier=art_classifier, confidence=0.0, targeted=False, learning_rate=1e-1, max_iter=20,
                    binary_search_steps=10, initial_const=1e-3, abort_early=True, use_resize=False, 
                    use_importance=False, nb_parallel=2, batch_size=1, variable_h=0.2)

    # Generate adversarial samples with ART Zeroth Order Optimization attack
    x_train_adv = attacker.generate(test_X_i)

    return x_test_adv

In [73]:
x_test_adv = get_adversarial_examples(model,X_test_std.values,Y_test,EPS = 0.1,model_type = 'xbg',attack = 'ZOO')

ZOO:   0%|          | 0/1066 [00:00<?, ?it/s]

In [70]:
acc = []
for i in range(1):
    test_X_i, test_Y_i = extract_10dB(X_test_std.values,Y_test)
    test_Y_i_hat = art_classifier.predict(x_test_adv)
    accuracy = np.sum(np.argmax(test_Y_i_hat, axis=1) == test_Y_i) / len(test_Y_i)
    print("Accuracy on adversarial test examples: {}%".format(accuracy * 100))   
    acc.append(accuracy)

Accuracy on adversarial test examples: 46.15384615384615%
